# 3 - Neural Machine Translation by Jointly Learning to Align and Translate

해당 내용은 16년도 [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473) 논문의 내용의 컨셉을 구현한 것임.

## Introduction

2장에서 'information compressions'을 조금 완화시키고자 context vector를 decoder의 매 step마다 넣어주었다.

하지만 이것으로 source sentence의 모든 정보를 담기에는 아직 부족하다. 그래서 이번장에서는 **attention** 을 사용해서 decoding step 마다 entire source sentence를 볼 수 있게 한다.

Attention은 가중치와 유사하게 0 ~ 1 사이 값을 가지고 합이 1이 되는 attention vector $a$를 구한 뒤에 hidden state $H$와 weighted sum을 하게 된다.

$w = \Sigma _i a_i h_i$

## Data Preparation

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

import en_core_web_sm, de_core_news_sm

spacy_de = de_core_news_sm.load()
spacy_en = en_core_web_sm.load()

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(
tokenize=tokenize_de,
init_token='<sos>',
eos_token='<eos>',
lower=True
)

TRG = Field(
    tokenize=tokenize_en,
    init_token='<sos>',
    eos_token='<eos>',
    lower=True
)

train_data, valid_data, test_data = Multi30k.splits(
    exts=('.de', '.en'),
    fields=(SRC, TRG)
)

print(f'Number of training data : {len(train_data.examples)}')
print(f'Number of valid data : {len(valid_data.examples)}')
print(f'Number of test data : {len(test_data.examples)}')

SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

Number of training data : 29000
Number of valid data : 1014
Number of test data : 1000


## Building the Seq2Seq Model

### Encoder

<p align="center"><img src="../asset/3(1).png"></p>

저번의 single layer GRU와 동일하지만 이번엔 bidirectional RNN을 사용함.

하나의 layer마다 두개의 RNN이 있는데 *forward RNN*은 embedded sentence를 왼쪽에서 오른쪽, *backward RNN*은 그 반대 방향으로 작동한다.

`outputs`의 크기는 [src_len, batch_size, hid_dim * n_directions] 라서 세번째 axis에 forward RNN과 backward RNN의 hidden state가 포함된다.

Decoder는 bidirectional 이 아니고 single context vecotr 만 필요하기 때문에 forward, backward hidden state를 concatenate 해줘서 사용한다

In [3]:
class Encoder(nn.Module):
    def __init__(self ,input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        '''
        src - [src_len, batch_size]
        embedded - [src_len, batch_size, emb_dim]
        outputs = [src_len, batch_size, hid_dim * num_directions]
        hidden = [n_layers * num_directions, batch_size, hid_dim]
        '''

        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2, :, ], hidden[-1, :, :]), dim=1)))

        return outputs, hidden

## Attention

입력으로 decoder의 previous hidden state $s_{t-1}$와 encoder에서 얻어진 forward, backward hidden state가 모두 stacked된 $H$를 받는다. 

출력으로는 attention vector $a_t$를 내뱉고, vector 길이는 source sentence와 같다.

### Attention vector 계산

1. $s_{t-1}$와 $H$ 사이의 정보를 계산한다.

$E_t = tanh(attn(s_{t-1}, H))$

이때 `attn`은 linear layer를 의미함

위의 식을 통해 각각의 encoder hidden state와 previous hidden state가 얼마나 match 되는지 계산할 수 있다.

2. 얻어진 $E_t$는 [dec_hid_dim, src_len] 인데 이를 [src_len]으로 차원 맞춰줌

$\hat{a}_t = vE_t$

이때 $v$는 hidden state에서 어떤 지점(토큰)을 중요하게 볼것인지 결정해주는 weight이라고 생각할 수 있음

3. 이후 attention vector 내부 값들이 0~1 사이의 값이고, 합해서 1이 되도록 softmax에 넣는다

$a_t = softmax(\hat{a}_t)$


<p align="center"><img src="../asset/3(2).png"></p>

In [4]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        '''
        hidden = [batch_size, dec_hid_dim]
        encoder_outputs - [src_len, batch_size, enc_hid_dim * 2]
        '''
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        # decoder hidden state를 attentino 계산을 위해 src_len 만큼 반복해서 늘려줌

        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1) # -> [batch_size, src_len, dec_hid_dim]
        encoder_outputs = encoder_outputs.permute(1, 0, 2) # -> [batch_size, src_len, enc_hid_dim * 2]

        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2))) # [batch_size, src_len, dec_hid_dim]
        attention = self.v(energy).squeeze(2) # -> [batch_size, src_len]

        return F.softmax(attention, dim=1)

## Decoder

Decoder는 위에서 구현한 `attention`을 가지고 있는데, 입력으로 previous hidden state $s_{t-1}$와 모든 stacked encoder hidden state $H$를 받아서 attention vector $a_t$를 생성한다.

그다음 attention vector를 가중치로 활용하여 새로운 $H$를 생성한다.

$w_t = a_tH$

이후 Decoder RNN에 embedded input $d(y_t)$, weighted source vector $w_t$, 전 단계 hidden state $s_{t-1}$ 가 들어가서 새로운 hidden state $s_t$를 추출한다.

이후 다음 단어 예측을 위해 linear layer $f$에 현 단계 hidden state와, weighted vector, embedded input이 들어간다.

<p align="center"><img src="../asset/3(3).png"></p>

In [5]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        '''
        input - [batch_size]
        hidden - [batch_size, dec_hid_dim]
        encoder_outputs = [src_len, batch_size, enc_hid_dim * 2] 
        '''

        input = input.unsqueeze(0) # -> [1, batch_size]
        embedded = self.dropout(self.embedding(input)) # [1, batch_size, emb_dim]
        a = self.attention(hidden, encoder_outputs) # -> [batch_size, src_len]
        a = a.unsqueeze(1) # -> [batch_size, 1, src_len]

        encoder_outputs = encoder_outputs.permute(1, 0, 2) # -> [batch_size, src_len, enc_hid_dim * 2]
        
        # `torch.bmm` : batch matrix multiplicatoin
        weighted = torch.bmm(a, encoder_outputs) # -> [batch_size, 1, enc_hid_dim * 2]
        weighted = weighted.permute(1, 0, 2) # -> [1, batch_size, enc_hid_dim * 2]
        rnn_input = torch.cat((embedded, weighted), dim=2) # -> [1, batch_size, emb_dim + enc_hid_dim * 2]

        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))

        assert (output == hidden).all()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))

        return prediction, hidden.squeeze(0)

In [6]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        '''
        src - [src_len, batch_size]
        trg - [trg_len, batch_size]
        '''
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        input = trg[0, :]

        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)

            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1)

            input = trg[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

In [7]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7854, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,518,661 trainable parameters


In [9]:
optimizer = optim.Adam(model.parameters())

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [10]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [11]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [12]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Training하면서 validation error

In [13]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '3-seq2seq-model.pt')
    
    print(f'Epoch : {epoch+1:02} | Time : {epoch_mins}m {epoch_secs}s | Train loss : {train_loss:.5f} | Valid loss : {valid_loss:.5f}')

Epoch : 01 | Time : 0m 40s | Train loss : 5.03217 | Valid loss : 4.85520
Epoch : 02 | Time : 0m 39s | Train loss : 4.19268 | Valid loss : 4.80471
Epoch : 03 | Time : 0m 39s | Train loss : 3.58479 | Valid loss : 3.82384
Epoch : 04 | Time : 0m 40s | Train loss : 3.00107 | Valid loss : 3.48618
Epoch : 05 | Time : 0m 39s | Train loss : 2.58933 | Valid loss : 3.31239
Epoch : 06 | Time : 0m 39s | Train loss : 2.28571 | Valid loss : 3.24563
Epoch : 07 | Time : 0m 39s | Train loss : 2.03876 | Valid loss : 3.15424
Epoch : 08 | Time : 0m 39s | Train loss : 1.81303 | Valid loss : 3.23614
Epoch : 09 | Time : 0m 39s | Train loss : 1.64232 | Valid loss : 3.23125
Epoch : 10 | Time : 0m 39s | Train loss : 1.52104 | Valid loss : 3.35088


In [14]:
model.load_state_dict(torch.load('3-seq2seq-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.160 | Test PPL:  23.565 |


: 